# Hudson Valley Gentrification and Changes in Farming

## Notebook 3: USDA Agriculture Census data

In [343]:
import pandas as pd
import requests
import os
import geopandas as gpd

In [344]:
api_key_nass = os.getenv("QUICK_STATS_API_KEY")

In [345]:
gdf_zipcodes = gpd.read_file("data/zctas_core.geojson")

In [346]:
gdf_zipcodes.head(1)

,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,12577,12577,B5,G6350,S,19999901,169531,+41.4214832,-074.1168149,"POLYGON ((288257.82 142661.193, 288349.37 1426..."


In [347]:
zipcodes = gdf_zipcodes.ZCTA5CE20.values

In [348]:
zipcodes

array(['12577', '10953', '10930', '12158', '10990', '12054', '10950',
       '12143', '12529', '12586', '12589', '10980', '10509', '12545',
       '12543', '12582', '12729', '10933', '12518', '06757', '06068',
       '06784', '06069', '06039', '12123', '12169', '12427', '12015',
       '12423', '12444', '01254', '01237', '01258', '18458', '18405',
       '18461', '18417', '18415', '18435', '18336', '18340', '01267',
       '01230', '01201', '01266', '07422', '07827', '07456', '07418',
       '07462', '07461', '07421', '12590', '12062', '12168', '12018',
       '12468', '12482', '12042', '12569', '12424', '12473', '12553',
       '10969', '12460', '10975', '12538', '12138', '12153', '12061',
       '12603', '12413', '12051', '12480', '12439', '12485', '12463',
       '12581', '12734', '12448', '12787', '12428', '12726', '12762',
       '12751', '12421', '12513', '12173', '12195', '12498', '12490',
       '12457', '12477', '12791', '12741', '12776', '12733', '12481',
       '12494', '127

## Download Economics and Demographics sectors from Agriculture Census and identify relevant variables

In [349]:
source_desc = "CENSUS"
sector_desc = ["Economics", "Demographics"]
group_desc = ['EXPENSES','FARMS & LAND & ASSETS','INCOME','PRODUCERS']
# These are the columns, in order, that (within a given sector) sequentially identify variables at any given level
group_cols = ["group_desc","commodity_desc", "class_desc", "prodn_practice_desc", "statisticcat_desc", "unit_desc","domain_desc","domaincat_desc"]

### Check whether there are fewer than 50,000 results at each specified year/level/sector

In [350]:
num_results = []
for year in ["2017","2022"]:
    for level in ["ZIP CODE", 'COUNTY']:
        for sector in sector_desc:
            r = requests.get(url=f'https://quickstats.nass.usda.gov/api/get_counts/?key={api_key_nass}', params={"sector_desc": sector, 
                                                                                                            "year": year, 
                                                                                                            "state_fips_code": "36", 
                                                                                                            "agg_level_desc": level}).json()
            num_results.append((year, level, sector, r['count']))

In [351]:
num_results

[('2017', 'ZIP CODE', 'Economics', 24414),
 ('2017', 'ZIP CODE', 'Demographics', 19578),
 ('2017', 'COUNTY', 'Economics', 24958),
 ('2017', 'COUNTY', 'Demographics', 17316),
 ('2022', 'ZIP CODE', 'Economics', 23609),
 ('2022', 'ZIP CODE', 'Demographics', 18113),
 ('2022', 'COUNTY', 'Economics', 24697),
 ('2022', 'COUNTY', 'Demographics', 13755)]

### Perform API calls for specified year/aggregation/sector; store results and transform into dataframes

In [352]:
ag_census_results = {}
for year in ["2017","2022"]:
    ag_census_results[year] = {}
    for level in ["zip code", 'county']:
        ag_census_results[year][level] = {}
        for sector in sector_desc:
            r = requests.get(url=f"https://quickstats.nass.usda.gov/api/api_GET/?key={api_key_nass}", 
                            params={"sector_desc": sector, 
                                    "year": year, 
                                    "state_fips_code": "36", 
                                    "agg_level_desc": level}).json()
            ag_census_results[year][level][sector] = r

In [353]:
for year in ["2017","2022"]:
    for level in ["zip code", 'county']:
        for sector in sector_desc:
            ag_census_results[year][level][sector]['df'] = pd.DataFrame.from_dict(ag_census_results[year][level][sector]['data'])

In [354]:
ag_census_results['2017']['county']['Economics']['df'].group_desc.unique()

array(['ENERGY', 'EXPENSES', 'FARMS & LAND & ASSETS', 'INCOME'],
      dtype=object)

In [355]:
for col in ag_census_results['2017']['county']['Economics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1).columns:
    print(col)
    print(ag_census_results['2017']['county']['Economics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1)[col].unique())

group_desc
['ENERGY' 'EXPENSES' 'FARMS & LAND & ASSETS' 'INCOME']
commodity_desc
['ENERGY' 'AG SERVICES' 'ANIMAL TOTALS' 'CHEMICAL TOTALS' 'DEPRECIATION'
 'EXPENSE TOTALS' 'FEED' 'FERTILIZER TOTALS' 'FUELS' 'INTEREST' 'LABOR'
 'RENT' 'SEEDS' 'SEEDS & PLANTS TOTALS' 'SUPPLIES & REPAIRS' 'TAXES'
 'AG LAND' 'FARM OPERATIONS' 'GOVT PROGRAMS' 'LAND AREA'
 'MACHINERY TOTALS' 'MACHINERY, OTHER' 'PRACTICES' 'SELF PROPELLED'
 'TRACTORS' 'TRUCKS' 'CCC LOANS' 'COMMODITY TOTALS' 'INCOME, FARM-RELATED'
 'INCOME, NET CASH FARM']
class_desc
['RENEWABLE, HARVEST BIOMASS FOR PRODUCTION'
 'CUSTOM SERVICES FOR LIVESTOCK, INCL MEDICAL SUPPLIES & VETERINARY'
 'CUSTOMWORK' 'MACHINERY RENTAL' 'OTHER' 'UTILITIES' '(EXCL BREEDING)'
 'ALL CLASSES' 'BREEDING' 'OPERATING' 'OPERATING, PAID BY LANDLORD'
 'INCL LIME & SOIL CONDITIONERS' 'INCL LUBRICANTS' 'NON-REAL ESTATE'
 'REAL ESTATE' 'CONTRACT' 'HIRED' 'HIRED, GE 150 DAYS'
 'HIRED, LT 150 DAYS' 'MIGRANT' 'UNPAID' 'CASH, CROPLAND'
 'CASH, LAND & BUILDINGS' 'CASH, 

In [356]:
for col in ag_census_results['2017']['county']['Demographics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1).columns:
    print(col)
    print(ag_census_results['2017']['county']['Demographics']['df'].groupby(group_cols).size().reset_index().drop(0, axis=1)[col].unique())

group_desc
['ANIMAL TOTALS' 'AQUACULTURE' 'CROP TOTALS' 'DAIRY' 'EXPENSES'
 'FARMS & LAND & ASSETS' 'FIELD CROPS' 'FRUIT & TREE NUTS' 'HORTICULTURE'
 'INCOME' 'LIVESTOCK' 'POULTRY' 'PRODUCERS' 'SPECIALTY' 'VEGETABLES']
commodity_desc
['ANIMAL TOTALS' 'AQUACULTURE TOTALS' 'CROP TOTALS' 'MILK'
 'EXPENSE TOTALS' 'LABOR' 'AG LAND' 'FARM OPERATIONS' 'INTERNET'
 'PRACTICES' 'BARLEY' 'CORN' 'FIELD CROPS, OTHER' 'GRAIN' 'HAY & HAYLAGE'
 'OATS' 'SOYBEANS' 'WHEAT' 'APPLES' 'FRUIT & TREE NUT TOTALS' 'GRAPES'
 'PEACHES' 'CUT CHRISTMAS TREES'
 'CUT CHRISTMAS TREES & SHORT TERM WOODY TREES' 'HORTICULTURE TOTALS'
 'NURSERY TOTALS' 'SOD' 'COMMODITY TOTALS' 'GOVT PROGRAMS'
 'INCOME, FARM-RELATED' 'INCOME, NET CASH FARM' 'CATTLE' 'GOATS' 'HOGS'
 'SHEEP' 'SHEEP & GOATS TOTALS' 'CHICKENS' 'POULTRY TOTALS' 'TURKEYS'
 'PRODUCERS' 'PRODUCERS, PRINCIPAL' 'EQUINE' 'SPECIALTY ANIMAL TOTALS'
 'POTATOES' 'SQUASH' 'SWEET CORN' 'VEGETABLE TOTALS']
class_desc
['INCL PRODUCTS' 'ALL CLASSES' 'OPERATING' 'HIRED'
 '(EXC

In [357]:
# save raw data to CSVs
for year in ["2017","2022"]:
    for level in ["zip code", 'county']:
        for sector in sector_desc:
            ag_census_results[year][level][sector]['df'].to_csv(f"data/ag_census_{"".join(level.split(" "))}_{year}_{sector}.csv", index=False, encoding='utf-8')

In [358]:
# save variables to CSVs
for year in ["2017","2022"]:
    for level in ["zip code", 'county']:
        for sector in sector_desc:
            df = ag_census_results[year][level][sector]['df'].groupby(group_cols).size().reset_index().drop(0, axis=1)
            df.to_csv(f"data/ag_census_var_csvs/ag_census_{"".join(level.split(" "))}_{year}_{sector}_variables.csv", index=False, encoding='utf-8')

Select relevant variables and prepare data for analysis

In [359]:
# Make separate dataframe for each universe and year

econ2017_zipcode_df = ag_census_results['2017']['zip code']['Economics']['df']
econ2022_zipcode_df = ag_census_results['2022']['zip code']['Economics']['df']
econ2017_county_df = ag_census_results['2017']['county']['Economics']['df']
econ2022_county_df = ag_census_results['2022']['county']['Economics']['df']

demog2017_zipcode_df = ag_census_results['2017']['zip code']['Demographics']['df']
demog2022_zipcode_df = ag_census_results['2022']['zip code']['Demographics']['df']
demog2017_county_df = ag_census_results['2017']['county']['Demographics']['df']
demog2022_county_df = ag_census_results['2022']['county']['Demographics']['df']

In [360]:
# Select just variables that I want to include in my study

econ2017_zipcode_df_subset = econ2017_zipcode_df.query("group_desc in ['FARMS & LAND & ASSETS','INCOME'] and commodity_desc in ['AG LAND','COMMODITY TOTALS'] and class_desc in ['CROPLAND, HARVESTED','ALL CLASSES'] and prodn_practice_desc != 'ORGANIC'").copy()
econ2022_zipcode_df_subset = econ2022_zipcode_df.query("group_desc in ['FARMS & LAND & ASSETS','INCOME'] and commodity_desc in ['AG LAND','COMMODITY TOTALS'] and class_desc in ['CROPLAND, HARVESTED','ALL CLASSES'] and prodn_practice_desc != 'ORGANIC'").copy()

In [361]:
# Select just variables that I want to include in my study

demog2017_zipcode_df_subset = demog2017_zipcode_df.query("domain_desc == 'TENURE' or prodn_practice_desc == 'PRIMARY OCCUPATION, FARMING'").copy()
demog2022_zipcode_df_subset = demog2022_zipcode_df.query("domain_desc == 'TENURE' or prodn_practice_desc == 'PRIMARY OCCUPATION, FARMING'").copy()

In [362]:
# For zipcode dataframes, make a smaller set just limited to zipcodes in target area

econ2017_zipcode_df_core = econ2017_zipcode_df_subset.query("zip_5 in @zipcodes").copy()
econ2022_zipcode_df_core = econ2022_zipcode_df_subset.query("zip_5 in @zipcodes").copy()
demog2017_zipcode_df_core = demog2017_zipcode_df_subset.query("zip_5 in @zipcodes").copy()
demog2022_zipcode_df_core = demog2022_zipcode_df_subset.query("zip_5 in @zipcodes").copy()

In [363]:
# Select just variables that I want to include in my study

econ2017_county_df_subset = econ2017_county_df.query("((((group_desc == 'EXPENSES' or group_desc == 'FARMS & LAND & ASSETS') and (commodity_desc in ['LABOR', 'AG LAND', 'LAND AREA']) and unit_desc != 'OPERATIONS' and (domain_desc == 'TOTAL' or domain_desc == 'LABOR') and (class_desc == 'HIRED' or class_desc == 'CROPLAND' or class_desc == 'INCL BUILDINGS' or class_desc == 'INCL NON-AG' or class_desc == 'CONTRACT') and unit_desc != '$ / OPERATION') or (group_desc == 'INCOME' and (commodity_desc == 'COMMODITY TOTALS' or commodity_desc == 'INCOME, NET CASH FARM') and unit_desc == '$' and domaincat_desc == 'NOT SPECIFIED')) and prodn_practice_desc == 'ALL PRODUCTION PRACTICES') or class_desc == 'CASH, LAND & BUILDINGS' or (group_desc == 'FARMS & LAND & ASSETS' and prodn_practice_desc == 'ALL PRODUCTION PRACTICES' and statisticcat_desc == 'OPERATIONS' and domain_desc == 'TOTAL')").copy()
econ2022_county_df_subset = econ2022_county_df.query("((((group_desc == 'EXPENSES' or group_desc == 'FARMS & LAND & ASSETS') and (commodity_desc in ['LABOR', 'AG LAND', 'LAND AREA']) and unit_desc != 'OPERATIONS' and (domain_desc == 'TOTAL' or domain_desc == 'LABOR') and (class_desc == 'HIRED' or class_desc == 'CROPLAND' or class_desc == 'INCL BUILDINGS' or class_desc == 'INCL NON-AG' or class_desc == 'CONTRACT') and unit_desc != '$ / OPERATION') or (group_desc == 'INCOME' and (commodity_desc == 'COMMODITY TOTALS' or commodity_desc == 'INCOME, NET CASH FARM') and unit_desc == '$' and domaincat_desc == 'NOT SPECIFIED')) and prodn_practice_desc == 'ALL PRODUCTION PRACTICES') or class_desc == 'CASH, LAND & BUILDINGS' or (group_desc == 'FARMS & LAND & ASSETS' and prodn_practice_desc == 'ALL PRODUCTION PRACTICES' and statisticcat_desc == 'OPERATIONS' and domain_desc == 'TOTAL')").copy()

In [364]:
# Select just variables that I want to include in my study

demog2017_county_df_subset = demog2017_county_df.query("((group_desc in ['FARMS & LAND & ASSETS','PRODUCERS'] and commodity_desc in ['AG LAND','FARM OPERATIONS','PRODUCERS'] and domain_desc in ['TENURE','TOTAL'] and unit_desc in ['PRODUCERS','ACRES','YEARS'] and class_desc in ['ALL CLASSES','(ALL)','CROPLAND, HARVESTED','AGE 25 TO 34','AGE 35 TO 44','AGE 45 TO 54','AGE 55 TO 64','AGE 65 TO 74','AGE GE 75','AGE LT 25'] and prodn_practice_desc in ['OWNED, IN FARMS','RENTED FROM OTHERS, IN FARMS','ALL PRODUCTION PRACTICES','PRIMARY OCCUPATION, FARMING','YEARS ON ANY OPERATION, 6 TO 10 YEARS','YEARS ON ANY OPERATION, GE 11 YEARS','YEARS ON ANY OPERATION, LT 11 YEARS','YEARS ON ANY OPERATION, LT 6 YEARS','YEARS ON PRESENT OPERATION, 3 TO 4 YEARS','YEARS ON PRESENT OPERATION, 5 TO 9 YEARS','YEARS ON PRESENT OPERATION, GE 10 YEARS','YEARS ON PRESENT OPERATION, LT 3 YEARS','RESIDENCE, NOT ON OPERATION','RESIDENCE, ON OPERATION'])) or statisticcat_desc == 'YEARS ON ANY OPERATION, AVG'").copy()
demog2022_county_df_subset = demog2022_county_df.query("((group_desc in ['FARMS & LAND & ASSETS','PRODUCERS'] and commodity_desc in ['AG LAND','FARM OPERATIONS','PRODUCERS'] and domain_desc in ['TENURE','TOTAL'] and unit_desc in ['PRODUCERS','ACRES','YEARS'] and class_desc in ['ALL CLASSES','(ALL)','CROPLAND, HARVESTED','AGE 25 TO 34','AGE 35 TO 44','AGE 45 TO 54','AGE 55 TO 64','AGE 65 TO 74','AGE GE 75','AGE LT 25'] and prodn_practice_desc in ['OWNED, IN FARMS','RENTED FROM OTHERS, IN FARMS','ALL PRODUCTION PRACTICES','PRIMARY OCCUPATION, FARMING','YEARS ON ANY OPERATION, 6 TO 10 YEARS','YEARS ON ANY OPERATION, GE 11 YEARS','YEARS ON ANY OPERATION, LT 11 YEARS','YEARS ON ANY OPERATION, LT 6 YEARS','YEARS ON PRESENT OPERATION, 3 TO 4 YEARS','YEARS ON PRESENT OPERATION, 5 TO 9 YEARS','YEARS ON PRESENT OPERATION, GE 10 YEARS','YEARS ON PRESENT OPERATION, LT 3 YEARS','RESIDENCE, NOT ON OPERATION','RESIDENCE, ON OPERATION'])) or statisticcat_desc == 'YEARS ON ANY OPERATION, AVG'").copy()

In [365]:
# Join variable subsets together by zipcode/county level and year

county2017_df = pd.concat([econ2017_county_df_subset,demog2017_county_df_subset]).copy().reset_index(drop=True)
county2022_df = pd.concat([econ2022_county_df_subset,demog2022_county_df_subset]).copy().reset_index(drop=True)
zipcode2017_df = pd.concat([econ2017_zipcode_df_core,demog2017_zipcode_df_core]).copy().reset_index(drop=True)
zipcode2022_df = pd.concat([econ2022_zipcode_df_core,demog2022_zipcode_df_core]).copy().reset_index(drop=True)

Cast 'Value' and 'CV (%)' columns as numbers, and replace text placeholders with approximate values.

From the [NASS QuickStats glossary](https://quickstats.nass.usda.gov/src/glossary.pdf):
* CV (%) = Coefficient of variation = Ratio of the standard error to the estimate, expressed as a percent.
* (D) = Withheld to avoid disclosing data for individual operations.
* (H) = Coefficient of variation or generalized coefficient of variation is greater than or equal to 99.95 percent or the standard error is greater than or equal to 99.95 percent of the mean.
* (L) = Coefficient of variation or generalized coefficient of variation is less than 0.05 percent or the standard error is less than 0.05 percent of the mean.

In [366]:
for df in [county2017_df,county2022_df,zipcode2017_df,zipcode2022_df]:
    df['Value'] = df['Value'].str.replace(',', '')
    df['Value'] = pd.to_numeric(df.Value, errors='coerce')
    df['CV_pct'] = pd.to_numeric(df['CV (%)'], errors='coerce')
    for idx, row in df.iterrows():
        if row['CV (%)'] == '(H)':
            df.loc[idx, 'CV_pct'] = 100
        elif row['CV (%)'] == '(L)':
            df.loc[idx, 'CV_pct'] = 0
        # Leave (D) as None, since data was withheld
    df.drop('CV (%)', axis=1, inplace=True)

2022 Economics has 'LABOR: (1 TO 4 HIRED WORKERS)' instead of three separate groups ('LABOR: (1 HIRED WORKERS)', 'LABOR: (2 HIRED WORKERS)', 'LABOR: (3 TO 4 HIRED WORKERS)') in `domaincat_desc`. Sum those three for 2017, and average `CV (%)`.

In [367]:
workers2017 = county2017_df.query("domaincat_desc in ['LABOR: (1 HIRED WORKERS)', 'LABOR: (2 HIRED WORKERS)', 'LABOR: (3 TO 4 HIRED WORKERS)']").groupby('county_code').sum()['Value']
workers2017_cv = county2017_df.query("domaincat_desc in ['LABOR: (1 HIRED WORKERS)', 'LABOR: (2 HIRED WORKERS)', 'LABOR: (3 TO 4 HIRED WORKERS)']").groupby('county_code').mean('CV_pct')['CV_pct']

for idx, row in county2017_df.iterrows():
    if row.domaincat_desc in ['LABOR: (1 HIRED WORKERS)', 'LABOR: (2 HIRED WORKERS)', 'LABOR: (3 TO 4 HIRED WORKERS)']:
        county2017_df.loc[idx, 'domaincat_desc'] = 'LABOR: (1 TO 4 HIRED WORKERS)'
        aggval = workers2017.loc[row.county_code]
        cvval = workers2017_cv.loc[row.county_code]
        
        county2017_df.loc[idx, 'Value'] = aggval
        county2017_df.loc[idx, 'CV_pct'] = cvval


In [368]:
print(len(county2017_df))
county2017_df.drop_duplicates(inplace=True)
print(len(county2017_df))

3080
2969


Identify which variables are not repeated across years.

In [369]:
dfs = [county2017_df,county2022_df,zipcode2017_df,zipcode2022_df]

In [370]:
county2017_vars = set()
county2022_vars = set()
for idx, row in county2017_df.iterrows():
    var = tuple(row[col] for col in group_cols)
    #print(var)
    if var not in county2017_vars:
        county2017_vars.add(var)
    
for idx, row in county2022_df.iterrows():
    var = tuple(row[col] for col in group_cols)
    #print(var)
    if var not in county2022_vars:
        county2022_vars.add(var)

In [371]:
len(county2017_vars ^ county2022_vars)

2

In [372]:
# variable(s) in 2022 county level that aren't in 2017 county level
county2022_vars.difference(county2017_vars)

{('PRODUCERS',
  'PRODUCERS',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'PRODUCERS',
  'PRODUCERS',
  'TOTAL',
  'NOT SPECIFIED')}

In [373]:
# variable(s) in 2017 county level that aren't in 2022 county level
county2017_vars.difference(county2022_vars)

{('PRODUCERS',
  'PRODUCERS, PRINCIPAL',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'YEARS ON ANY OPERATION, AVG',
  'YEARS',
  'TOTAL',
  'NOT SPECIFIED')}

In [374]:
county_vars = county2017_vars & county2022_vars

In [375]:
len(county_vars)

49

In [376]:
zipcode2017_vars = set()
zipcode2022_vars = set()
for idx, row in zipcode2017_df.iterrows():
    var = tuple(row[col] for col in group_cols)
    #print(var)
    if var not in zipcode2017_vars:
        zipcode2017_vars.add(var)
    
for idx, row in zipcode2022_df.iterrows():
    var = tuple(row[col] for col in group_cols)
    #print(var)
    if var not in zipcode2022_vars:
        zipcode2022_vars.add(var)

In [377]:
len(zipcode2017_vars ^ zipcode2022_vars)

0

In [378]:
zipcode_vars = zipcode2017_vars & zipcode2022_vars

In [379]:
len(zipcode_vars)

12

Investigate non-matched `ALL CLASSES` variable to see if its values are the same as `(ALL)`:
`('PRODUCERS',
  'PRODUCERS',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'PRODUCERS',
  'PRODUCERS',
  'TOTAL',
  'NOT SPECIFIED')`

In [380]:
county2022_df[county2022_df.commodity_desc.str.contains('PRODUCERS')][group_cols].drop_duplicates()

,group_desc,commodity_desc,class_desc,prodn_practice_desc,statisticcat_desc,unit_desc,domain_desc,domaincat_desc
1546,PRODUCERS,PRODUCERS,(ALL),ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1608,PRODUCERS,PRODUCERS,AGE 25 TO 34,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1666,PRODUCERS,PRODUCERS,AGE 35 TO 44,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1728,PRODUCERS,PRODUCERS,AGE 45 TO 54,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1787,PRODUCERS,PRODUCERS,AGE 55 TO 64,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1847,PRODUCERS,PRODUCERS,AGE 65 TO 74,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1909,PRODUCERS,PRODUCERS,AGE GE 75,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1968,PRODUCERS,PRODUCERS,AGE LT 25,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
2021,PRODUCERS,PRODUCERS,ALL CLASSES,ALL PRODUCTION PRACTICES,"AGE, AVG",YEARS,TOTAL,NOT SPECIFIED
2083,PRODUCERS,PRODUCERS,ALL CLASSES,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED


In [381]:
county2017_df[county2017_df.commodity_desc.str.contains('PRODUCERS')][group_cols].drop_duplicates()

,group_desc,commodity_desc,class_desc,prodn_practice_desc,statisticcat_desc,unit_desc,domain_desc,domaincat_desc
1648,PRODUCERS,PRODUCERS,(ALL),ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1709,PRODUCERS,PRODUCERS,AGE 25 TO 34,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1766,PRODUCERS,PRODUCERS,AGE 35 TO 44,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1827,PRODUCERS,PRODUCERS,AGE 45 TO 54,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1886,PRODUCERS,PRODUCERS,AGE 55 TO 64,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
1946,PRODUCERS,PRODUCERS,AGE 65 TO 74,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
2005,PRODUCERS,PRODUCERS,AGE GE 75,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
2065,PRODUCERS,PRODUCERS,AGE LT 25,ALL PRODUCTION PRACTICES,PRODUCERS,PRODUCERS,TOTAL,NOT SPECIFIED
2117,PRODUCERS,PRODUCERS,ALL CLASSES,ALL PRODUCTION PRACTICES,"AGE, AVG",YEARS,TOTAL,NOT SPECIFIED
2178,PRODUCERS,PRODUCERS,ALL CLASSES,ALL PRODUCTION PRACTICES,"YEARS ON ANY OPERATION, AVG",YEARS,TOTAL,NOT SPECIFIED


In [382]:
len(county2022_df.county_code.unique())

62

In [383]:
# Limit dataset to just the two possibly-identical variables and see if they have duplicate values: if so, should be twice the number of counties

len(county2022_df.assign(duplicate = county2022_df[county2022_df.commodity_desc.str.contains('PRODUCERS')].query("class_desc in ['ALL CLASSES', '(ALL)'] and statisticcat_desc == 'PRODUCERS' and prodn_practice_desc == 'ALL PRODUCTION PRACTICES'").duplicated(subset=[col for col in group_cols if col != 'class_desc'])).query("duplicate == True")) + 1

124

Assign names to each variable that occurs in both years and drop values with no variable name

In [384]:
county_vars_dict = {}
county_i = range(len(county_vars))
county_zip = zip(county_i,county_vars)
for i, var in county_zip:
    county_vars_dict[f"county_var{i}"] = var
county_vars_dict


{'county_var0': ('EXPENSES',
  'LABOR',
  'CONTRACT',
  'ALL PRODUCTION PRACTICES',
  'EXPENSE',
  '$',
  'TOTAL',
  'NOT SPECIFIED'),
 'county_var1': ('EXPENSES',
  'LABOR',
  'HIRED',
  'ALL PRODUCTION PRACTICES',
  'EXPENSE',
  '$',
  'TOTAL',
  'NOT SPECIFIED'),
 'county_var2': ('PRODUCERS',
  'PRODUCERS',
  'AGE 45 TO 54',
  'ALL PRODUCTION PRACTICES',
  'PRODUCERS',
  'PRODUCERS',
  'TOTAL',
  'NOT SPECIFIED'),
 'county_var3': ('FARMS & LAND & ASSETS',
  'FARM OPERATIONS',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'AREA OPERATED',
  'ACRES',
  'TENURE',
  'TENURE: (PART OWNER)'),
 'county_var4': ('PRODUCERS',
  'PRODUCERS',
  'AGE 55 TO 64',
  'ALL PRODUCTION PRACTICES',
  'PRODUCERS',
  'PRODUCERS',
  'TOTAL',
  'NOT SPECIFIED'),
 'county_var5': ('INCOME',
  'COMMODITY TOTALS',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'SALES',
  '$',
  'TOTAL',
  'NOT SPECIFIED'),
 'county_var6': ('PRODUCERS',
  'PRODUCERS',
  'AGE 35 TO 44',
  'ALL PRODUCTION PRACTICES',
  'PRODU

In [385]:
zipcode_vars_dict = {}
zipcode_i = range(len(zipcode_vars))
zipcode_zip = zip(zipcode_i,zipcode_vars)
for i, var in zipcode_zip:
    zipcode_vars_dict[f"zipcode_var{i}"] = var
zipcode_vars_dict

{'zipcode_var0': ('FARMS & LAND & ASSETS',
  'FARM OPERATIONS',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'OPERATIONS',
  'OPERATIONS',
  'TENURE',
  'TENURE: (PART OWNER)'),
 'zipcode_var1': ('INCOME',
  'COMMODITY TOTALS',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'SALES',
  'OPERATIONS',
  'FARM SALES',
  'FARM SALES: (50,000 TO 249,999 $)'),
 'zipcode_var2': ('FARMS & LAND & ASSETS',
  'AG LAND',
  'CROPLAND, HARVESTED',
  'ALL PRODUCTION PRACTICES',
  'AREA',
  'OPERATIONS',
  'AREA HARVESTED',
  'AREA HARVESTED: (50 TO 499 ACRES)'),
 'zipcode_var3': ('INCOME',
  'COMMODITY TOTALS',
  'ALL CLASSES',
  'ALL PRODUCTION PRACTICES',
  'SALES',
  'OPERATIONS',
  'FARM SALES',
  'FARM SALES: (250,000 OR MORE $)'),
 'zipcode_var4': ('FARMS & LAND & ASSETS',
  'AG LAND',
  'CROPLAND, HARVESTED',
  'ALL PRODUCTION PRACTICES',
  'AREA',
  'OPERATIONS',
  'AREA HARVESTED',
  'AREA HARVESTED: (1.0 TO 49.9 ACRES)'),
 'zipcode_var5': ('INCOME',
  'COMMODITY TOTALS',
  'ALL CLASSES

In [386]:
zipcode2017_df['var_name'] = ''
for idx, row in zipcode2017_df.iterrows():
    var = tuple([row[col] for col in group_cols])
    for key, val in zipcode_vars_dict.items():
        if val == var:
            zipcode2017_df.loc[idx, 'var_name'] = key

In [387]:
zipcode2022_df['var_name'] = ''
for idx, row in zipcode2022_df.iterrows():
    var = tuple([row[col] for col in group_cols])
    for key, val in zipcode_vars_dict.items():
        if val == var:
            zipcode2022_df.loc[idx, 'var_name'] = key

In [388]:
county2017_df['var_name'] = ''
for idx, row in county2017_df.iterrows():
    var = tuple([row[col] for col in group_cols])
    for key, val in county_vars_dict.items():
        if val == var:
            county2017_df.loc[idx, 'var_name'] = key

In [389]:
county2022_df['var_name'] = ''
for idx, row in county2022_df.iterrows():
    var = tuple([row[col] for col in group_cols])
    for key, val in county_vars_dict.items():
        if val == var:
            county2022_df.loc[idx, 'var_name'] = key

In [390]:
county2017_df = county2017_df.query("var_name != ''").copy()
county2022_df = county2022_df.query("var_name != ''").copy()

In [391]:
for key, val in zipcode_vars_dict.items():
    zipcode_vars_dict[key] = dict(zip(group_cols, val))

zipcode_vars_dict

{'zipcode_var0': {'group_desc': 'FARMS & LAND & ASSETS',
  'commodity_desc': 'FARM OPERATIONS',
  'class_desc': 'ALL CLASSES',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'OPERATIONS',
  'unit_desc': 'OPERATIONS',
  'domain_desc': 'TENURE',
  'domaincat_desc': 'TENURE: (PART OWNER)'},
 'zipcode_var1': {'group_desc': 'INCOME',
  'commodity_desc': 'COMMODITY TOTALS',
  'class_desc': 'ALL CLASSES',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'SALES',
  'unit_desc': 'OPERATIONS',
  'domain_desc': 'FARM SALES',
  'domaincat_desc': 'FARM SALES: (50,000 TO 249,999 $)'},
 'zipcode_var2': {'group_desc': 'FARMS & LAND & ASSETS',
  'commodity_desc': 'AG LAND',
  'class_desc': 'CROPLAND, HARVESTED',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'AREA',
  'unit_desc': 'OPERATIONS',
  'domain_desc': 'AREA HARVESTED',
  'domaincat_desc': 'AREA HARVESTED: (50 TO 499 ACRES)'},
 'zipcode_var3': {'group_desc': '

In [392]:
for key, val in zipcode_vars_dict.items():
    if " ".join(val.values()) in ['FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS AREA HARVESTED AREA HARVESTED: (1.0 TO 49.9 ACRES)','FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS AREA HARVESTED AREA HARVESTED: (50 TO 499 ACRES)','FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS AREA HARVESTED AREA HARVESTED: (500 OR MORE ACRES)','FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS TOTAL NOT SPECIFIED','INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS FARM SALES FARM SALES: (250,000 OR MORE $)','INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS FARM SALES FARM SALES: (50,000 TO 249,999 $)','INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS FARM SALES FARM SALES: (LESS THAN 50,000 $)','INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS TOTAL NOT SPECIFIED']:
        print(key)

zipcode_var1
zipcode_var2
zipcode_var3
zipcode_var4
zipcode_var5
zipcode_var6
zipcode_var10
zipcode_var11


In [393]:
for key, val in zipcode_vars_dict.items():
    description = ""
    var_name = ""
    if " ".join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS AREA HARVESTED AREA HARVESTED: (1.0 TO 49.9 ACRES)':
        description = "Number small farms (1.0 to 49.9 acres harvested cropland)"
        var_name = "number_small_farms"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS AREA HARVESTED AREA HARVESTED: (50 TO 499 ACRES)':
        description = "Number medium farms (50 to 499 acres harvested cropland)"
        var_name = "number_medium_farms"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS AREA HARVESTED AREA HARVESTED: (500 OR MORE ACRES)':
        description = "Number large farms (500 or more acres harvested cropland)"
        var_name = "number_large_farms"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA OPERATIONS TOTAL NOT SPECIFIED':
        description = "Total farm operations harvesting"
        var_name = "total_farms_harvesting"
    elif ' '.join(val.values()) == 'INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS FARM SALES FARM SALES: (250,000 OR MORE $)':
        description = "High-sales farm operations (250,000 or MORE $)"
        var_name = "number_high_sales_farms"
    elif ' '.join(val.values()) == 'INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS FARM SALES FARM SALES: (50,000 TO 249,999 $)':
        description = "Medium-sales farm operations (50,000 to 249,999 $)"
        var_name = "number_medium_sales_farms"
    elif ' '.join(val.values()) == 'INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS FARM SALES FARM SALES: (LESS THAN 50,000 $)':
        description = "Low-sales farm operations (less than 50,000 $)"
        var_name = "number_low_sales_farms"
    elif ' '.join(val.values()) == 'INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES OPERATIONS TOTAL NOT SPECIFIED':
        description = "Total farm operations selling commodities"
        var_name = "total_farms_selling"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS FARM OPERATIONS ALL CLASSES ALL PRODUCTION PRACTICES OPERATIONS OPERATIONS TENURE TENURE: (FULL OWNER)':
        description = "Number full-owner farm operators"
        var_name = "number_full_owner_operators"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS FARM OPERATIONS ALL CLASSES ALL PRODUCTION PRACTICES OPERATIONS OPERATIONS TENURE TENURE: (PART OWNER)':
        description = "Number part-owner farm operators"
        var_name = "number_part_owner_operators"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS FARM OPERATIONS ALL CLASSES ALL PRODUCTION PRACTICES OPERATIONS OPERATIONS TENURE TENURE: (TENANT)':
        description = "Number tenant farm operators"
        var_name = "number_tenant_operators"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES PRIMARY OCCUPATION, FARMING PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Total producers with farming as primary occupation"
        var_name = "total_full_time_farmers"
    else:
        print(key)
    if var_name != '':
        zipcode_vars_dict[key]['var_name'] = var_name
    if description != '':
        zipcode_vars_dict[key]['description'] = description


In [394]:
zipcode_vars_dict

{'zipcode_var0': {'group_desc': 'FARMS & LAND & ASSETS',
  'commodity_desc': 'FARM OPERATIONS',
  'class_desc': 'ALL CLASSES',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'OPERATIONS',
  'unit_desc': 'OPERATIONS',
  'domain_desc': 'TENURE',
  'domaincat_desc': 'TENURE: (PART OWNER)',
  'var_name': 'number_part_owner_operators',
  'description': 'Number part-owner farm operators'},
 'zipcode_var1': {'group_desc': 'INCOME',
  'commodity_desc': 'COMMODITY TOTALS',
  'class_desc': 'ALL CLASSES',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'SALES',
  'unit_desc': 'OPERATIONS',
  'domain_desc': 'FARM SALES',
  'domaincat_desc': 'FARM SALES: (50,000 TO 249,999 $)',
  'var_name': 'number_medium_sales_farms',
  'description': 'Medium-sales farm operations (50,000 to 249,999 $)'},
 'zipcode_var2': {'group_desc': 'FARMS & LAND & ASSETS',
  'commodity_desc': 'AG LAND',
  'class_desc': 'CROPLAND, HARVESTED',
  'prodn_practice_desc': 'A

In [395]:
for key, val in county_vars_dict.items():
    county_vars_dict[key] = dict(zip(group_cols, val))

In [396]:
for key, val in county_vars_dict.items():
    description = ""
    var_name = ""

    if ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND ALL CLASSES OWNED, IN FARMS AREA ACRES TENURE TENURE: (PART OWNER)':
        description = "Acres owned by part-owners"
        var_name = "acres_owned_part_owners"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND ALL CLASSES RENTED FROM OTHERS, IN FARMS AREA ACRES TENURE TENURE: (PART OWNER)':
        description = "Acres rented by part-owners"
        var_name = "acres_rented_part_owners"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA ACRES TENURE TENURE: (FULL OWNER)':
        description = "Cropland harvested by full owners (in acres)"
        var_name = "acres_harvested_full_owners"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA ACRES TENURE TENURE: (PART OWNER)':
        description = "Cropland harvested by part-owners (in acres)"
        var_name = "acres_harvested_part_owners"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND, HARVESTED ALL PRODUCTION PRACTICES AREA ACRES TENURE TENURE: (TENANT)':
        description = "Cropland harvested by tenants (in acres)"
        var_name = "acres_harvested_tenants"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS FARM OPERATIONS ALL CLASSES ALL PRODUCTION PRACTICES AREA OPERATED ACRES TENURE TENURE: (FULL OWNER)':
        description = "Area operated by full owners (in acres)"
        var_name = "acres_operated_full_owners"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS FARM OPERATIONS ALL CLASSES ALL PRODUCTION PRACTICES AREA OPERATED ACRES TENURE TENURE: (PART OWNER)':
        description = "Area operated by part-owners (in acres)"
        var_name = "acres_operated_part_owners"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS FARM OPERATIONS ALL CLASSES ALL PRODUCTION PRACTICES AREA OPERATED ACRES TENURE TENURE: (TENANT)':
        description = "Area operated by tenants (in acres)"
        var_name = "acres_operated_tenants"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS (ALL) ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Total producers"
        var_name = "total_producers"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS AGE 25 TO 34 ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers (age 25-34)"
        var_name = "producers_agecat_2"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS AGE 35 TO 44 ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers (age 35-44)"
        var_name = "producers_agecat_3"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS AGE 45 TO 54 ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers (age 45-54)"
        var_name = "producers_agecat_4"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS AGE 55 TO 64 ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers (age 55-64)"
        var_name = "producers_agecat_5"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS AGE 65 TO 74 ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers (age 65-74)"
        var_name = "producers_agecat_6"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS AGE GE 75 ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers (age 75 and above)"
        var_name = "producers_agecat_7"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS AGE LT 25 ALL PRODUCTION PRACTICES PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers (age 24 and below)"
        var_name = "producers_agecat_1"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES ALL PRODUCTION PRACTICES AGE, AVG YEARS TOTAL NOT SPECIFIED':
        description = "Producers, avg. age"
        var_name = "producers_avg_age"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES RESIDENCE, NOT ON OPERATION PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers not housed on operation"
        var_name = "housed_off_farm"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES RESIDENCE, ON OPERATION PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Producers housed on operation"
        var_name = "housed_on_farm"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES YEARS ON ANY OPERATION, 6 TO 10 YEARS PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Mid-career producers (6-10 years)"
        var_name = "producers_mid_career"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES YEARS ON ANY OPERATION, GE 11 YEARS PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Late-career producers (11+ years)"
        var_name = "producers_late_career"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES YEARS ON ANY OPERATION, LT 6 YEARS PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Early-career producers (5 or fewer years)"
        var_name = "producers_early_career"
    elif ' '.join(val.values()) == 'EXPENSES LABOR CONTRACT ALL PRODUCTION PRACTICES EXPENSE $ TOTAL NOT SPECIFIED':
        description = "Total expenses for contract labor ($)"
        var_name = "total_contract_labor_expenses"
    elif ' '.join(val.values()) == 'EXPENSES LABOR HIRED ALL PRODUCTION PRACTICES EXPENSE $ TOTAL NOT SPECIFIED':
        description = "Total expenses for hired labor ($)"
        var_name = "total_hired_labor_expenses"
    # elif ' '.join(val.values()) == 'EXPENSES LABOR HIRED ALL PRODUCTION PRACTICES WORKERS NUMBER LABOR LABOR: (1 HIRED WORKERS)':
    #     description = "Number of hired workers: 1"
    #     var_name = "hired_labor_cat_1"
    elif ' '.join(val.values()) == 'EXPENSES LABOR HIRED ALL PRODUCTION PRACTICES WORKERS NUMBER LABOR LABOR: (1 TO 4 HIRED WORKERS)':
        description = "Number of hired workers: 1-4"
        var_name = "hired_labor_cat_1"
    elif ' '.join(val.values()) == 'EXPENSES LABOR HIRED ALL PRODUCTION PRACTICES WORKERS NUMBER LABOR LABOR: (5 TO 9 HIRED WORKERS)':
        description = "Number of hired workers: 5-9"
        var_name = "hired_labor_cat_2"
    elif ' '.join(val.values()) == 'EXPENSES LABOR HIRED ALL PRODUCTION PRACTICES WORKERS NUMBER LABOR LABOR: (10 OR MORE HIRED WORKERS)':
        description = "Number of hired workers: 10+"
        var_name = "hired_labor_cat_3"
    elif ' '.join(val.values()) == 'EXPENSES LABOR HIRED ALL PRODUCTION PRACTICES WORKERS NUMBER TOTAL NOT SPECIFIED':
        description = "Total number of hired workers"
        var_name = "total_hired_labor"
    elif ' '.join(val.values()) == 'EXPENSES RENT CASH, LAND & BUILDINGS ALL PRODUCTION PRACTICES EXPENSE $ TOTAL NOT SPECIFIED':
        description = "Total rent costs (land and buildings)"
        var_name = "rent_costs_dollars"
    elif ' '.join(val.values()) == 'EXPENSES RENT CASH, LAND & BUILDINGS ALL PRODUCTION PRACTICES EXPENSE OPERATIONS TOTAL NOT SPECIFIED':
        description = "Total operations renting land and buildings"
        var_name = "rent_burdened_operations"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND CROPLAND ALL PRODUCTION PRACTICES AREA ACRES TOTAL NOT SPECIFIED':
        description = "Total cropland (in acres)"
        var_name = "total_acres_cropland"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND INCL BUILDINGS ALL PRODUCTION PRACTICES ASSET VALUE $ / ACRE TOTAL NOT SPECIFIED':
        description = "Total asset value of land and buildings ($/acre)"
        var_name = "total_farm_value_per_acre"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS LAND AREA INCL NON-AG ALL PRODUCTION PRACTICES AREA ACRES TOTAL NOT SPECIFIED':
        description = "Total farmland area, including non-agricultural (in acres)"
        var_name = "total_acres_farmland"
    elif ' '.join(val.values()) == 'INCOME COMMODITY TOTALS ALL CLASSES ALL PRODUCTION PRACTICES SALES $ TOTAL NOT SPECIFIED':
        description = "Total income from commodity sales ($)"
        var_name = "total_income"
    elif ' '.join(val.values()) == 'INCOME INCOME, NET CASH FARM OF OPERATIONS ALL PRODUCTION PRACTICES NET INCOME $ TOTAL NOT SPECIFIED':
        description = "Net income of operations ($)"
        var_name = "net_income_operations"
    elif ' '.join(val.values()) == 'INCOME INCOME, NET CASH FARM OF PRODUCERS ALL PRODUCTION PRACTICES NET INCOME $ TOTAL NOT SPECIFIED':
        description = "Net income of producers ($)"
        var_name = "net_income_producers"
    elif ' '.join(val.values()) == 'PRODUCERS PRODUCERS ALL CLASSES PRIMARY OCCUPATION, FARMING PRODUCERS PRODUCERS TOTAL NOT SPECIFIED':
        description = "Total producers with farming as primary occupation"
        var_name = "total_full_time_farmers"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS AG LAND INCL BUILDINGS ALL PRODUCTION PRACTICES ASSET VALUE $ TOTAL NOT SPECIFIED':
        description = "Total asset value of land and buildings ($)"
        var_name = "total_farm_value"
    elif ' '.join(val.values()) == "PRODUCERS PRODUCERS ALL CLASSES ALL PRODUCTION PRACTICES YEARS ON ANY OPERATION, AVG YEARS TOTAL NOT SPECIFIED":
        description = "Producers avg. career length"
        var_name = "producers_avg_career"
    elif ' '.join(val.values()) == 'FARMS & LAND & ASSETS FARM OPERATIONS ALL CLASSES ALL PRODUCTION PRACTICES OPERATIONS OPERATIONS TOTAL NOT SPECIFIED':
        description = "Total farm operations"
        var_name = "total_farms"
    else:
        print(key)
        print(' '.join(val.values()))
    if var_name != '':
        county_vars_dict[key]['var_name'] = var_name
    if description != '':
        county_vars_dict[key]['description'] = description


county_var16
PRODUCERS PRODUCERS ALL CLASSES YEARS ON PRESENT OPERATION, LT 3 YEARS PRODUCERS PRODUCERS TOTAL NOT SPECIFIED
county_var17
PRODUCERS PRODUCERS ALL CLASSES YEARS ON PRESENT OPERATION, 5 TO 9 YEARS PRODUCERS PRODUCERS TOTAL NOT SPECIFIED
county_var18
PRODUCERS PRODUCERS ALL CLASSES YEARS ON ANY OPERATION, LT 11 YEARS AREA OPERATED ACRES TOTAL NOT SPECIFIED
county_var19
EXPENSES LABOR HIRED ALL PRODUCTION PRACTICES EXPENSE $ LABOR LABOR: (HIRED WORKERS GE 150 DAYS & LT 150 DAYS)
county_var24
INCOME COMMODITY TOTALS INCL VALUE-ADDED ALL PRODUCTION PRACTICES SALES $ TOTAL NOT SPECIFIED
county_var28
PRODUCERS PRODUCERS ALL CLASSES YEARS ON ANY OPERATION, LT 11 YEARS PRODUCERS PRODUCERS TOTAL NOT SPECIFIED
county_var36
PRODUCERS PRODUCERS ALL CLASSES ALL PRODUCTION PRACTICES YEARS ON PRESENT OPERATION, AVG YEARS TOTAL NOT SPECIFIED
county_var37
PRODUCERS PRODUCERS ALL CLASSES YEARS ON PRESENT OPERATION, GE 10 YEARS PRODUCERS PRODUCERS TOTAL NOT SPECIFIED
county_var48
PRODUCERS P

In [397]:
county_vars_dict

{'county_var0': {'group_desc': 'EXPENSES',
  'commodity_desc': 'LABOR',
  'class_desc': 'CONTRACT',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'EXPENSE',
  'unit_desc': '$',
  'domain_desc': 'TOTAL',
  'domaincat_desc': 'NOT SPECIFIED',
  'var_name': 'total_contract_labor_expenses',
  'description': 'Total expenses for contract labor ($)'},
 'county_var1': {'group_desc': 'EXPENSES',
  'commodity_desc': 'LABOR',
  'class_desc': 'HIRED',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'EXPENSE',
  'unit_desc': '$',
  'domain_desc': 'TOTAL',
  'domaincat_desc': 'NOT SPECIFIED',
  'var_name': 'total_hired_labor_expenses',
  'description': 'Total expenses for hired labor ($)'},
 'county_var2': {'group_desc': 'PRODUCERS',
  'commodity_desc': 'PRODUCERS',
  'class_desc': 'AGE 45 TO 54',
  'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
  'statisticcat_desc': 'PRODUCERS',
  'unit_desc': 'PRODUCERS',
  'domain_desc': 'TOTAL',
  'do

In [398]:
county2017_df.var_name.unique()

array(['county_var0', 'county_var19', 'county_var1', 'county_var47',
       'county_var38', 'county_var20', 'county_var11', 'county_var23',
       'county_var39', 'county_var21', 'county_var27', 'county_var41',
       'county_var29', 'county_var10', 'county_var5', 'county_var24',
       'county_var31', 'county_var12', 'county_var13', 'county_var43',
       'county_var14', 'county_var40', 'county_var9', 'county_var25',
       'county_var3', 'county_var22', 'county_var32', 'county_var45',
       'county_var6', 'county_var2', 'county_var4', 'county_var44',
       'county_var8', 'county_var42', 'county_var26', 'county_var35',
       'county_var36', 'county_var7', 'county_var15', 'county_var34',
       'county_var30', 'county_var46', 'county_var18', 'county_var28',
       'county_var33', 'county_var48', 'county_var17', 'county_var37',
       'county_var16'], dtype=object)

In [399]:
for i, df in enumerate([zipcode2017_df, zipcode2022_df, county2017_df, county2022_df]):
    if i < 2:
        level = "zipcode"
    else:
        level = "county"
    df['description'] = ''
    for idx, row in df.iterrows():
        key = row.var_name
        vardict = eval(f"{level}_vars_dict")
        new_var_name = vardict[key].get('var_name')
        description = vardict[key].get("description")
        if new_var_name is not None:
            df.at[idx, 'var_name'] = new_var_name
            df.at[idx, 'description'] = description

zipcode2017_df.head()[["var_name", 'description']]

,var_name,description
0,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
1,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
2,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
3,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
4,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...


In [400]:
zipcode2017_df.head()

,state_alpha,unit_desc,watershed_desc,Value,load_time,statisticcat_desc,prodn_practice_desc,group_desc,begin_code,class_desc,...,domain_desc,country_name,zip_5,region_desc,state_name,asd_code,source_desc,CV_pct,var_name,description
0,NY,OPERATIONS,,7,2018-02-01 12:00:00.000,AREA,ALL PRODUCTION PRACTICES,FARMS & LAND & ASSETS,00,"CROPLAND, HARVESTED",...,AREA HARVESTED,UNITED STATES,10509,,NEW YORK,,CENSUS,NaN,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
1,NY,OPERATIONS,,7,2018-02-01 12:00:00.000,AREA,ALL PRODUCTION PRACTICES,FARMS & LAND & ASSETS,00,"CROPLAND, HARVESTED",...,AREA HARVESTED,UNITED STATES,10512,,NEW YORK,,CENSUS,NaN,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
2,NY,OPERATIONS,,4,2018-02-01 12:00:00.000,AREA,ALL PRODUCTION PRACTICES,FARMS & LAND & ASSETS,00,"CROPLAND, HARVESTED",...,AREA HARVESTED,UNITED STATES,10516,,NEW YORK,,CENSUS,NaN,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
3,NY,OPERATIONS,,11,2018-02-01 12:00:00.000,AREA,ALL PRODUCTION PRACTICES,FARMS & LAND & ASSETS,00,"CROPLAND, HARVESTED",...,AREA HARVESTED,UNITED STATES,10524,,NEW YORK,,CENSUS,NaN,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...
4,NY,OPERATIONS,,4,2018-02-01 12:00:00.000,AREA,ALL PRODUCTION PRACTICES,FARMS & LAND & ASSETS,00,"CROPLAND, HARVESTED",...,AREA HARVESTED,UNITED STATES,10535,,NEW YORK,,CENSUS,NaN,number_small_farms,Number small farms (1.0 to 49.9 acres harveste...


In [401]:
county2022_df.to_csv("data/agcensus_county_2022.csv", index=False, encoding='utf-8')
county2017_df.to_csv("data/agcensus_county_2017.csv", index=False, encoding='utf-8')
zipcode2017_df.to_csv("data/agcensus_zipcode_2017.csv", index=False, encoding='utf-8')
zipcode2022_df.to_csv("data/agcensus_zipcode_2022.csv", index=False, encoding='utf-8')

In [402]:
import json

with open("data/county_vars.json", "w") as countyjsonfile:
    json.dump(county_vars_dict, countyjsonfile)

with open("data/zipcode_vars.json", "w") as zipcodejsonfile:
    json.dump(zipcode_vars_dict, zipcodejsonfile)
